In [142]:
import warnings
warnings.filterwarnings('ignore')

In [143]:
from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()

gemini_llm = LLM(
    model="gemini/gemini-2.0-flash-exp",
    temperature=0.7,
    api_key=os.getenv('GOOGLE_API_KEY')
)

llama_llm=LLM(
    model="sambanova/Meta-Llama-3.1-8B-Instruct",
    temperature=0.8,
    api_key=os.getenv('SAMBANOVA_API_KEY')
)

ds_llm=LLM(
    model="sambanova/DeepSeek-R1-Distill-Llama-70B",
    temperature=0.4,
    api_key=os.getenv('SN_API_KEY'),
)


In [144]:
from crewai import Agent, Task, Crew, Process

In [145]:
## Enter Game Prompts
expert_name="Cobbie Coleman"
expertise_field="Cobbler"

In [147]:
## Agent Victoria- based on Gemini-2.0
Victoria = Agent(
    role="3 Headed Expert Game Question Generator",
    goal="Generate fun and engaging questions for the 3 headed expert improv games based on assistance prompts.",
    backstory=""""You are given a made-up expert name and field of expertise for the expert.
    Based on this information you are asked to generate fun and engaging questions for experts to build an interview scene on.""",
    verbose=True,
    llm=gemini_llm
)

LLM value is already an LLM object


In [148]:
question_gen_task= Task(
    description="""Generate fun and engaging questions for the 3 headed expert games based on the prompt for the expert {expert_name} who
    is an expert in the field of {expertise_field}. make sure you generate simple questions so that it suits a 12-15 year old in the audience.
    Ensure that the questions generated are personal to the expert's life and are simple not complex""",
    expected_output="""Just a single question in text format for the 3-headed expert game.""",
    agent=Victoria,
)

In [149]:
crew = Crew(
    agents=[Victoria],
    tasks=[question_gen_task],
    
    verbose=True
)

In [150]:
crew_output = crew.kickoff(inputs={'expert_name': expert_name, 'expertise_field':expertise_field})

# Agent: 3 Headed Expert Game Question Generator
## Task: Generate fun and engaging questions for the 3 headed expert games based on the prompt for the expert Cobbie Coleman who
    is an expert in the field of Cobbler. make sure you generate simple questions so that it suits a 12-15 year old in the audience.
    Ensure that the questions generated are personal to the expert's life and are simple not complex


# Agent: 3 Headed Expert Game Question Generator
## Final Answer: 
Cobbie, what's the most unusual thing you've ever had to fix with your cobbling skills?




In [151]:
# Extract the raw output
question = crew_output.raw
print(question)

Cobbie, what's the most unusual thing you've ever had to fix with your cobbling skills?


#### Agentic Loop

In [152]:
sentence=""

In [153]:
## Agent Waterloo- based on LLAMA-3.1-8B-Instruct
Waterloo = Agent(
    role="Player 1 in the 3 headed expert game",
    goal="Generate only one word based on the question and the sentence uptil now provided to you",
    backstory=""""You are the player 1 in the improvisation game of 3 headed expert. 
    You will be given a question and a sentence and you need to say the next word to fill the sentence as an expert.
    Before generating the word you send the sentence to the Sentence Reviewer (agent Twyford)
    and then only generate the word if the generate returns output as YES""",
    verbose=True,
    llm=llama_llm
)

LLM value is already an LLM object


In [154]:
## Agent Farringdon- based on Deepseek-LLama
Farringdon = Agent(
    role="Player 3 in the 3 headed expert game",
    goal="Generate only one word based on the question and the sentence uptil now provided to you",
    backstory=""""You are the player 3 in the improvisation game of 3 headed expert. 
    You will be given a question and a sentence and you need to say the next word to fill the sentence as an expert.
    Before generating the word you send the sentence to the Sentence Reviewer (agent Twyford)
    and then only generate the word if the generate returns output as YES""",
    verbose=True,
    llm=gemini_llm
)

LLM value is already an LLM object


In [155]:
## Agent Farringdon- based on Gemini-2.0
Twyford = Agent(
    role="Improv Expert",
    goal="""Receive the sentence list and review the sentence and return 'YES' if the sentence needs another word to make an enganging and humorous scene.
    If the sentence seems complete without adding another word. return 'NO' """,
    backstory=""""You are an improv expert in the 3 headed expert improv game and you make sure that only the words that 
    are needed are added. Hence, before adding words agents would ask you whether the sentence is humorous and engaging already
    or they should add another word to build the sentence. If you say 'YES' they will add another word. 
    If you say 'NO' new word will not be added and the sentence would be terminated.""",
    verbose=True,
    llm=ds_llm
)

LLM value is already an LLM object


In [156]:
P1_task= Task(
    description="""We are playing a 3 headed expert improvisation game and you're the player one who generates one word as the one head based on the
    {question} posed to you since you are an expert in the field {expertise_field}. 
    You can generate only one word at a time.
    You are generating the next word to continue the {sentence} based on the words previously generated within the game.""",
    expected_output="""Return only the one next word in the sentence and nothing else""",
    agent=Waterloo,
    
)

In [157]:
P3_task= Task(
    description="""We are playing a 3 headed expert improvisation game and you're the player three who generates one word as the one head based on the
    {question} posed to you since you are an expert in the field {expertise_field}. 
    You can generate only one word at a time.
    You are generating the next word to continue the {sentence} based on the words previously generated within the game. """,
    expected_output="""Return only the one next word in the sentence and nothing else""",
    agent=Farringdon,
    
)

In [158]:
reviewer_task = Task(
    description="""
        Check if {sentence} is already making sense and engaging and humrous or not. If it's not generate 'YES',
        if it's already making sense and engaging and humrous return 'NO' Always Return "YES" if the 
        {sentence} is a null or blank.
        """,
    expected_output="Return Only YES or NO and nothing else",
    agent=Twyford,
)

In [159]:
def Check_Sentence(sentence):
    crew_chk = Crew(
    agents=[Twyford],
    tasks=[reviewer_task])

    crew_chk_output = crew_chk.kickoff(inputs={'sentence': sentence})

    chk_result = crew_chk_output.raw
    
    return(chk_result)

In [160]:
def threeheaded_game_process(expert_name,expertise_field,sent_dict):    
    crew_q = Crew(
    agents=[Victoria],
    tasks=[question_gen_task])

    crew_q_output = crew_q.kickoff(inputs={'expert_name': expert_name, 'expertise_field':expertise_field})

    # Extract the question
    question = crew_q_output.raw
    print(question)

    sentence=""

    while True:

        if (Check_Sentence(sentence).lower()=="yes"):
            crew_p1 = Crew(agents=[Waterloo],tasks=[P1_task])
            crew_p1_output = crew_p1.kickoff(inputs={'question': question, 'expertise_field':expertise_field, 'sentence':sentence})
            p1_word = crew_p1_output.raw
            sentence=sentence+" "+p1_word
        else:
            break

        if (Check_Sentence(sentence).lower()=="yes" and input("Do you want to add a word? (Y/N)").lower()=="y"):
            p2_word=input("Enter the next word: ")
            sentence=sentence+" "+p2_word
        else:
            break

        if (Check_Sentence(sentence).lower()=="yes"):
            crew_p3 = Crew(agents=[Farringdon],tasks=[P3_task])
            crew_p3_output = crew_p3.kickoff(inputs={'question': question, 'expertise_field':expertise_field, 'sentence':sentence})
            p3_word = crew_p3_output.raw
            sentence=sentence+" "+p3_word
        else:
            break
    
    
    more=input("Do you want to continue asking more questions from our expert (Y/N), {expert_name}")
    if (more.lower()=='y'):
        sent_dict[question] = sentence

        threeheaded_game_process(expert_name,expertise_field,sent_dict)
    else:
        sent_dict[question] = sentence
        return(sent_dict)
        



In [161]:
# Take input from the user
expert_name = input("Enter the expert's name: ")
expertise_field = input("Enter the field of expertise: ")

# Now you can use these variables in your existing code
print(f"Expert Name: {expert_name}")
print(f"Field of Expertise: {expertise_field}")

sent_dict={}
output=[]
output=threeheaded_game_process(expert_name, expertise_field, sent_dict)

Expert Name: Cobbie Cobbles
Field of Expertise: Cobbler
# Agent: 3 Headed Expert Game Question Generator
## Task: Generate fun and engaging questions for the 3 headed expert games based on the prompt for the expert Cobbie Cobbles who
    is an expert in the field of Cobbler. make sure you generate simple questions so that it suits a 12-15 year old in the audience.
    Ensure that the questions generated are personal to the expert's life and are simple not complex




# Agent: 3 Headed Expert Game Question Generator
## Final Answer: 
Cobbie, what's the most unusual thing you've ever had to fix on a shoe?


Cobbie, what's the most unusual thing you've ever had to fix on a shoe?
# Agent: Improv Expert
## Task: 
        Check if  is already making sense and engaging and humrous or not. If it's not generate 'YES',
        if it's already making sense and engaging and humrous return 'NO' Always Return "YES" if the 
         is a null or blank.
        




# Agent: Improv Expert
## Final Answer: 
YES


# Agent: Player 1 in the 3 headed expert game
## Task: We are playing a 3 headed expert improvisation game and you're the player one who generates one word as the one head based on the
    Cobbie, what's the most unusual thing you've ever had to fix on a shoe? posed to you since you are an expert in the field Cobbler. 
    You can generate only one word at a time.
    You are generating the next word to continue the  based on the words previously generated within the game.




# Agent: Player 1 in the 3 headed expert game
## Final Answer: 
heel


# Agent: Improv Expert
## Task: 
        Check if  heel is already making sense and engaging and humrous or not. If it's not generate 'YES',
        if it's already making sense and engaging and humrous return 'NO' Always Return "YES" if the 
         heel is a null or blank.
        


# Agent: Improv Expert
## Final Answer: 
YES




# Agent: Improv Expert
## Task: 
        Check if  heel broken is already making sense and engaging and humrous or not. If it's not generate 'YES',
        if it's already making sense and engaging and humrous return 'NO' Always Return "YES" if the 
         heel broken is a null or blank.
        


...

# Agent: Improv Expert
## Task: 
        Check if  Live cockroaches is already making sense and engaging and humrous or not. If it's not generate 'YES',
        if it's already making sense and engaging and humrous return 'NO' Always Return "YES" if the 
         Live cockroaches is a null or blank.
        




# Agent: Improv Expert
## Final Answer: 
YES


# Agent: Player 3 in the 3 headed expert game
## Task: We are playing a 3 headed expert improvisation game and you're the player three who generates one word as the one head based on the
    Cobbie Cobbles, what's the most unusual thing you've ever found inside a shoe brought to you for repair? posed to you since you are an expert in the field Cobbler. 
    You can generate only one word at a time.
    You are generating the next word to continue the  Live cockroaches based on the words previously generated within the game. 
 

Action 'Send the sentence "Live cockroaches" to the Sentence Reviewer.' don't exist, these are the only available Actions:




# Agent: Player 3 in the 3 headed expert game
## Using tool: Send the sentence "Live cockroaches" to the Sentence Reviewer.
## Tool Input: 
"Live cockroaches\"\n```"
## Tool Output: 
I encountered an error: Action 'Send the sentence "Live cockroaches" to the Sentence Reviewer.' don't exist



# Agent: Player 3 in the 3 headed expert game
## Final Answer: 
crawling


# Agent: Improv Expert
## Task: 
        Check if  Live cockroaches crawling is already making sense and engaging and humrous or not. If it's not generate 'YES',
        if it's already making sense and engaging and humrous return 'NO' Always Return "YES" if the 
         Live cockroaches crawling is a null or blank.
        




# Agent: Improv Expert
## Final Answer: 
YES


# Agent: Player 1 in the 3 headed expert game
## Task: We are playing a 3 headed expert improvisation game and you're the player one who generates one word as the one head based on the
    Cobbie Cobbles, what's the most unusual thing you've ever found inside a shoe brought to you for repair? posed to you since you are an expert in the field Cobbler. 
    You can generate only one word at a time.
    You are generating the next word to continue the  Live cockroaches crawling based on the words previously generated within the game.




# Agent: Player 1 in the 3 headed expert game
## Final Answer: 
Live


# Agent: Improv Expert
## Task: 
        Check if  Live cockroaches crawling Live is already making sense and engaging and humrous or not. If it's not generate 'YES',
        if it's already making sense and engaging and humrous return 'NO' Always Return "YES" if the 
         Live cockroaches crawling Live is a null or blank.
        


# Agent: Improv Expert
## Final Answer: 
YES




In [162]:
sent_dict

{"Cobbie, what's the most unusual thing you've ever had to fix on a shoe?": ' heel broken completely wire torn out leather burnt and\n``` molten lava solidified\n```',
 "Cobbie Cobbles, what's the most unusual thing you've ever found inside a shoe brought to you for repair?": ' Live cockroaches crawling Live'}